# Analyzing CIA Factbook Data Using SQL

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

# Overview of the Data

We'll begin by getting a sense of what the data looks like.

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

name - The name of the country.
area - The total land and sea area of the country.
population - The country's population.
population_growth- The country's population growth as a percentage.
birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
death_rate - The country's death rate, or the number of death a year per 1,000 people.
area- The country's total area (both land and water).
area_land - The country's land area in square kilometers.
area_water - The country's waterarea in square kilometers.
Let's start by calculating some summary statistics and see what they tell us.

# Summary Statistics

In [4]:
%%sql
SELECT MIN(population) AS min_population,
    MAX(population) as max_population,
    MIN(population_growth) as min_population_growth,
    MAX(population_growth) as max_population_growth
from facts

Done.


min_population,max_population,min_population_growth,max_population_growth
0,7256490011,0.0,4.02


A few things stick out from the summary statistics in the last screen:

There's a country with a population of 0
There's a country with a population of 7256490011 (or more than 7.2 billion people)
Let's use subqueries to zoom in on just these countries without using the specific values.

# Exploring Outliers

In [5]:
%%sql
SELECT name, population
  FROM facts
 where population = 0

Done.


name,population
Antarctica,0


It seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the CIA Factbook page for Antarctica:



In [6]:
%%sql
SELECT name, population
  FROM facts
 where population = 7256490011

Done.


name,population
World,7256490011


We also see that the table contains a row for the whole world, which explains the maximum population of over 7.2 billion we found earlier.

Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.

# Summary Statistics Revisited

In [7]:
%%sql
SELECT MIN(population) AS min_population,
    MAX(population) as max_population,
    MIN(population_growth) as min_population_growth,
    MAX(population_growth) as max_population_growth
from facts
where population != 7256490011 and population != 0

Done.


min_population,max_population,min_population_growth,max_population_growth
48,1367485388,0.0,4.02


There's a country whose population closes in on 1.4 billion!

# Exploring Average Population and Area

Let's explore density. Density depends on the population and the country's area. Let's look at the average values for these two columns.

We should take care of discarding the row for the whole planet.

In [8]:
%%sql
SELECT AVG(population) average_population,
       avg(area) as average_area
  FROM facts
where name != 'World'


Done.


average_population,average_area
32242666.56846473,555093.546184739


We see that the average population is around 32 million and the average area is 555 thousand square kilometers.

# Finding sparsely Populated Countries

Her I will be building on the query above to find countries that are sparsely populated. I will identify countries that have:

Below average values for population.
Above average values for area.

In [9]:
%%sql
SELECT *
FROM facts
where population <(SELECT AVG(population)
                   FROM facts
                   where name != 'World')
 and area > (SELECT avg(area)
             FROM facts
             where name != 'World')



Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46
9,as,Australia,7741220,7682300,58920,22751014,1.07,12.15,7.14,5.65
21,bl,Bolivia,1098581,1083301,15280,10800882,1.56,22.76,6.52,0.62
23,bc,Botswana,581730,566730,15000,2182719,1.21,20.96,13.39,4.56
34,ct,Central African Republic,622984,622984,0,5391539,2.13,35.08,13.8,0.0
36,ci,Chile,756102,743812,12290,17508260,0.82,13.83,6.0,0.34
87,kz,Kazakhstan,2724900,2699700,25200,18157122,1.14,19.15,8.21,0.41
100,ly,Libya,1759540,1759540,0,6411776,2.23,18.03,3.58,7.8
105,ma,Madagascar,587041,581540,5501,23812681,2.58,32.61,6.81,0.0
109,ml,Mali,1240192,1220190,20002,16955536,2.98,44.99,12.89,2.26


Some of these countries are generally known to be sparsely populated, so we have confidence in our results!

# Top 10 countries with the most people? 

In [13]:
%%sql
SELECT *
  FROM facts
where name != 'World'
Order by population DESC
Limit 10


Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44
77,in,India,3287263,2973193,314070,1251695584,1.22,19.55,7.32,0.04
197,ee,European Union,4324782,None,None,513949445,0.25,10.2,10.2,2.5
186,us,United States,9826675,9161966,664709,321368864,0.78,12.49,8.15,3.86
78,id,Indonesia,1904569,1811569,93000,255993674,0.92,16.72,6.37,1.16
24,br,Brazil,8515770,8358140,157630,204259812,0.77,14.46,6.58,0.14
132,pk,Pakistan,796095,770875,25220,199085847,1.46,22.58,6.49,1.54
129,ni,Nigeria,923768,910768,13000,181562056,2.45,37.64,12.9,0.22
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
143,rs,Russia,17098242,16377742,720500,142423773,0.04,11.6,13.69,1.69


The data presented here, has a high confidence to reality of population and not necessarily the exact population number.

This Countries has the highest population growth, hence the need to account for public utilities such as schools, hospitals and food security.

7 of the fastest growing populations are in Africa

# Countries with the highest migration rate?

In [23]:
%%sql
SELECT *
  FROM facts
where name != 'World'
Order by migration_rate DESC
Limit 10


Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
141,qa,Qatar,11586,11586,0,2194817,3.07,9.84,1.53,22.39
242,aq,American Samoa,199,199,0,54343,0.3,22.89,4.75,21.13
115,fm,"Micronesia, Federated States of",702,702,0,105216,0.46,20.54,4.23,20.93
170,sy,Syria,185180,183630,1550,17064854,0.16,22.17,4.0,19.79
176,tn,Tonga,747,717,30,106501,0.03,23.0,4.85,17.84
229,vi,British Virgin Islands,151,151,0,33454,2.32,10.91,4.99,17.28
103,lu,Luxembourg,2586,2586,0,570252,2.13,11.37,7.24,17.16
230,cj,Cayman Islands,264,264,0,56092,2.1,12.11,5.53,14.4
156,sn,Singapore,697,687,10,5674472,1.89,8.27,3.43,14.05
123,nr,Nauru,21,21,0,9540,0.55,24.95,5.87,13.63


In [ ]:
The countries above are experiencing the highest migration rate:
    
    If it's Immigration then those countries will experience population pressure in the society
    
    If it'

# Which countries have more water than land?

# Which countries will add the most people to their populations next year?

# Which countries have a higher death rate than birth rate?

# Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?